<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html wall_models.do.txt --ipynb_admon=hrule --without_solutions --no_abort -->
<!-- dom:TITLE: Uncertainty quantification and sensitivity analysis for arterial wall models -->

# Uncertainty quantification and sensitivity analysis for arterial wall models
**Vinzenz Gregor Eck**, Expert Analytics, Oslo  
**Jacob Sturdy**, Department of Structural Engineering, NTNU

Date: **Jul 1, 2021**

In [1]:
# ipython magic
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:

import matplotlib
import matplotlib.pyplot as plt

In [3]:
import chaospy as cp
import chaospy_wrapper as cpw
import numpy as np
from monte_carlo import generate_sample_matrices_mc
from monte_carlo import calculate_sensitivity_indices_mc
unit_mmhg_pa = 133.3
unit_pa_mmhg = 1./unit_mmhg_pa
unit_cm2_m2 = 1. / 100. / 100.
unit_m2_cm2 = 1. / unit_cm2_m2

# Introduction
<div id="sec:introduction"></div>
The arterial wall models we are investigating in this part, are used to describe the (visco-)elastic behaviour of arteries in one-dimensional
simulations of the cardiovascular system.

# Arterial Wall Models
<div id="sec:polynomialChaos"></div>

The elastic wall models are simplified algebraic functions $A(P)$ ([[eck2015stochastic;@boileau_benchmark_2015]](#eck2015stochastic;@boileau_benchmark_2015)),
which state the arterial lumen area $A$ as function of transmural pressure $P$.

For the calibration of the applied wall models, the wave speed in an arterial segment is required.
The wave speed is given from fluid dynamics equations for one-dimensional arteries ([[eck2015stochastic]](#eck2015stochastic)):

<!-- Equation labels as ordinary links -->
<div id="eq:defwaveSpeed"></div>

$$
\begin{equation}
 \tag{1}
c(P) = \sqrt{\frac{A(P)}{\rho\ C(P)}},
\end{equation}
$$

with blood density $\rho= 1050\ [kg\ m^{-3}]$ and compliance $C(P) = dA / dP$.

## *Quadratic* model
The *Quadratic* area-pressure relationship ([[sherwin2003]](#sherwin2003)) is defined as:

<!-- Equation labels as ordinary links -->
<div id="eq:lapArea"></div>

$$
\begin{equation}
A(P) = \left((P-P_s) \frac{A_s}{\lambda} + \sqrt{A_s} \right)^2,
 \tag{2}
\end{equation}
$$

where $\lambda$ is referred to as the stiffness coefficient and $A_s$
is the area at the reference pressure $P_s$.

The stiffness coefficient $\lambda$ is defined as:

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
\lambda = \frac{2 \rho c_s^2 A_s}{\sqrt{A_s}}
 \tag{3}
\end{equation}
$$

## *Logarithmic* model
The *Logarithmic* area-pressure relationship ([[Hayashi1980]](#Hayashi1980)) is defined as:

<!-- Equation labels as ordinary links -->
<div id="eq:expArea"></div>

$$
\begin{equation}
A(P) = A_s \left( 1 + \frac{1}{\beta} ln \left(\frac{P}{P_s}\right) \right)^2,
 \tag{4}
\end{equation}
$$

where $\beta$ is called the stiffness coefficient and $A_s$ is the
area at the reference pressure $P_s$.

The stiffness coefficient $\beta$ is defined as:

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}
\beta = \frac{2\ \rho c_s^2}{P_s}
 \tag{5}
\end{equation}
$$

## Exercise 1: Implement these two models in Python

Write a function that implements the relationship based on the following

In [4]:
def quadratic_area_model(pressure, parameters):
    # Implement the function here
    a_s, c_s, p_s, rho = parameters # Works when the first index of parameters indexes the parameters
    area = 0 # You need to implement this
    return area

See the solution if you need help.

## Exercise 2: Follow the 6 (7) steps of UQSA for this model

Perhaps you want to have a personalized or localized model of the arterial wall? How can you get values for $A_s$, $P_s$, $c$ and $\rho$? How much uncertainty do you have about these values? Do you imagine a different use case for this model? What uncertainties might be present in that use case?

* Step 1 Identification of the output(s) of interest Y

* Step 2 Identify the inputs of interest and the appropriate distribution for the situation you are interested in? Can you support your choice of distribution? (If you prefer to move on just assume some nominal 10\% uncertainty on the parameters. What does 10\% uncertainty mean? If you model this with a Normal random variable or Uniform random variable does it have a different meaning?).

* Step 3 Sampling of the input space to acquire samples of your inputs in a manner suited to your method for step 5

* Step 4 Evaluate the deterministic model at each sample point to obtain your output samples

* Step 5 Calculation of UQ and SA measures with your method of choice: Monte Carlo or Polynomial Chaos

* Step 6 Assess the convergence of UQ and SA measures

* Step 7 interpret the results

## Exercise 3: Evaluate your external model

If you have a model that is not written in python or needs to be evaluated offline. You can follow the same procedure with some minor modifications.

For Step 3 you will need to write these values out your model of interest. Do you need to generate an input file for each? Can you load a list of parameter values from a csv file? In any case you can generate a data file containing the sample points and then determine how to run your model for each sample in the data points.

For Step 4 you will use the output from step 3, and collect the output values in a similar data file.

For Step 5 you will load the results file from Step 4 and then proceed as if the values had been generated in Python.

In [5]:
# example sampling
u1 = cp.Uniform(0,1)
u2 = cp.Uniform(0,1)
joint_distribution = cp.J(u1, u2)
number_of_samples = 350
samples_random = joint_distribution.sample(size=number_of_samples, rule='R')

In [6]:
# example save samples to file
# Creates a csv file where each row corresponds to the sample number and each column with teh variables in the joint distribution
csv_file = "csv_samples.csv"
sep = '\t'
header = ["u1", "u2"]
header = sep.join(header)
np.savetxt(csv_file, samples_random, delimiter=sep, header=header)
# end example save samples to file

# generate external data
# load external samples
samples_random = np.genfromtxt(csv_file)

# evaluate model
ext_data = np.array([sample[0] + sample[1] + sample[0]*sample[1] for sample in samples_random.T])
header = ['y0']
header = sep.join(header)
filepath = "external_evaluations.csv"
np.savetxt(filepath, ext_data, delimiter=sep, header=header)
# end generate external data

# example load samples from file
# loads a csv file where the samples/or model evaluations for each sample are saved
# with one sample per row. Multiple components ofoutput can be stored as separate columns 
filepath = "external_evaluations.csv"
data = np.genfromtxt(filepath)

1. <div id="eck2015stochastic"></div> **V. G. Eck, J. Feinberg, H. P. Langtangen and L. R. Hellevik**.  Stochastic Sensitivity Analysis for Timing and Amplitude of Pressure waves in the Arterial System, *International Journal for Numerical Methods in Biomedical Engineering*, 31(4), 2015.

2. <div id="boileau_benchmark_2015"></div> **E. Boileau, P. Nithiarasu, P. J. Blanco, L. O. Mueller, F. E. Fossan, L. R. Hellevik, W. P. Donders, W. Huberts, M. Willemet and J. Alastruey**.  A benchmark study of numerical schemes for one-dimensional arterial blood flow modelling, *International Journal for Numerical Methods in Biomedical Engineering*, 31(10), pp. n/a-n/a, 2015.

3. <div id="sherwin2003"></div> **S. Sherwin, V. Franke, J. Peir\'o and K. Parker**.  One-Dimensional Modelling of Vascular Network in Space-Time Variables, *Journal of Engineering Mathematics*, 47(3-4), pp. 217-233, 2003.

4. <div id="Hayashi1980"></div> **K. Hayashi, H. Handa, S. Nagasawa, A. Okumura and K. Moritake**.  Stiffness and Elastic Behavior of Human Intracranial and Extracranial arteries, *Journal of Biomechanics*, 13(2), pp. 175-184, 1980.